In [1]:
import math
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from ipynb.fs.full.functions import i_star, i_c, i_s, properties, rotor_inlet, delta_star, lc_star, lc_OD, compressor_solver
from ipynb.fs.full.functions import delta_OD, rotor_outlet, stator_outlet, efficiency, checker, stage_solver, stage_solver_star

# Single-Stage Compressor

## Data

The compressor uses inlet guied vanes at the inlet of the compressor and the flow angle at the inlet of the compressor is 43 degrees. The blades are from type of C-4 circular arc and and also We have these informations:

In [2]:
alpha1 = 43                #degree

In [3]:
N_design = 6000            #RPM
N_stage  = 1

In [4]:
P01 = 100/101.325          #bar
T01 = 288                  #k

In [5]:
t_max_c = 0.12             #t_max/c

In [6]:
r_h1 = 0.2                 #meter
r_t1 = 0.3                 #meter
H1 = r_t1-r_h1

r_h2 = 0.205               #meter
r_t2 = 0.295               #meter
H2 = r_t2-r_h2

r_h3 = 0.21                #meter
r_t3 = 0.29                #meter
H3 = r_t3-r_h3

In [7]:
r_m1 = (r_h1+r_t1)/2
r_m2 = (r_h2+r_t2)/2
r_m3 = (r_h3+r_t3)/2

In [8]:
#for compressor:
C_p = 1.004                             #kJ/kg-k
gama = 1.4  
R = 0.287                               #kJ/kg-k

### Rotor

In [9]:
beta_prime1 = 50         #degree
beta_prime2 = 30         #degree
sigma_r = 1              #rotor solidity at r_m
c_r = 0.04               #rotor chord[meter]
s_r = c_r/sigma_r

### Stator

In [10]:
alpha_prime2 = 57.6      #degree
alpha_prime3 = 32.6      #degree
sigma_s = 0.91           #stator solidity at r_m
c_s = 0.05               #stator chord[meter]
s_s = c_s/sigma_s

In [11]:
blade_type = ['65_Series','circular']
stagger = (beta_prime1+beta_prime2)/2
a_c = 0.5

## Calculation Process

### i_star

In [12]:
theta = beta_prime1-beta_prime2

In [13]:
i_star(beta_prime1, stagger, a_c, theta, t_max_c, sigma_r, blade_type)

{'i_star': -0.15572863909250145,
 'alpha_star': 9.844271360907499,
 'beta1_star': 49.8442713609075,
 'E-Check': 0.0}

In [14]:
dic = i_star(beta_prime1, stagger, a_c, theta, t_max_c, sigma_r, blade_type)
i_star_r = dic['i_star']
alpha_star = dic['alpha_star']
beta1_star = dic['beta1_star']

## i_c

In [15]:
i_c(i_star_r, alpha_star, beta1_star, theta, stagger)

{'i_c': -9.463214370626282,
 'alpha_c': 0.5367856293737177,
 'beta1_c': 40.53678562937372,
 'E-Check': 1.0}

It is ok to have 'E-Check': 1.0. that's because of that We reduce i_c 1 degree for 2% more mass flow rate!!!

In [16]:
dic = i_c(i_star_r, alpha_star, beta1_star, theta, stagger)
i_c_r = dic['i_c']
alpha_c = dic['alpha_c']
beta1_c = dic['beta1_c']

## i_s

In [17]:
i_s(i_star_r, alpha_star, beta1_star, theta, stagger)

{'i_s': 8.171249657087026,
 'alpha_s': 18.171249657087024,
 'beta1_s': 58.17144681763575,
 'E-Check': 0.00019716054872276345}

In [18]:
dic = i_s(i_star_r, alpha_star, beta1_star, theta, stagger)
i_s_r = dic['i_s']
alpha_s = dic['alpha_s']
beta1_s = dic['beta1_s']

## Rotor Inlet

We calculate specifications at the rotor inlet for design parameters:

In [19]:
i = i_star_r
N = N_design              #RPM
sigma = sigma_r
beta1 = beta_prime1+i

In [20]:
df1_rotor_inlet_star, df2_rotor_inlet_star = rotor_inlet(r_h1, r_t1, N, beta1, alpha1, P01, T01, R)

In [21]:
df1_rotor_inlet_star

,P[bar],T[k],h[kj/kg],s[kj/kg-k]
Point 01,0.986923,288.000000,292.716203,5.866373
Point 1,0.926867,282.877435,287.573148,5.866373
Point 01_rel,1.004612,289.466164,294.188232,5.866373


In [22]:
df2_rotor_inlet_star

,m_dot[kg/sec],rho1[kg/m^3],C1[meter/sec],W1[meter/sec],C_m1[meter/sec],U1[meter/sec],rho1[kg/sec],A1[m^2],alpha1[degree],beta1[degree]
value,13.478038,1.156788,101.420463,115.022468,74.174231,157.079633,1.156788,0.15708,43,49.844271


In [23]:
W1_star = df2_rotor_inlet_star['W1[meter/sec]']['value']
T1_star = df1_rotor_inlet_star['T[k]']['Point 1']
M_W1_star = W1_star/((gama*R*1000*T1_star)**0.5)

In [24]:
M_W1_star

0.3411760695814368

## Rotor Outlet

### delta_star

In [25]:
C_m1 = df2_rotor_inlet_star['C_m1[meter/sec]']['value']
U1 = df2_rotor_inlet_star['U1[meter/sec]']['value']
h1 = df1_rotor_inlet_star['h[kj/kg]']['Point 1']

In [26]:
df1_rotor_outlet_star, df2_rotor_outlet_star = rotor_outlet(M_W1_star, df1_rotor_inlet_star, df2_rotor_inlet_star, gama, R, blade_type, i_s_r, i_c_r, 0, sigma, beta1, 0, i_star_r, i_star_r, alpha1, r_m1, r_m2, N_design, beta_prime2, H1, H2, N_design, t_max_c, a_c, s_r, theta)

In [27]:
df1_rotor_outlet_star

,P[bar],T[k],h[kj/kg],s[kj/kg-k]
Point 02,1.040851,292.598045,297.332640,5.867007
Point 2,0.943004,284.456151,289.158179,5.867007
Point 02_rel,1.002395,289.466164,294.188232,5.867007


In [28]:
df2_rotor_outlet_star

,m_dot[kg/sec],C2[meter/sec],W2[meter/sec],C_m2[meter/sec],U2[meter/sec],rho2[kg/m^3],A2[m^2],alpha2[degree],beta2[degree],E-Check
value,13.478038,127.862907,100.300082,81.457318,157.079633,1.170397,0.141372,50.426495,35.694837,0.000216


In [29]:
C_m2 = df2_rotor_outlet_star['C_m2[meter/sec]']['value']

In [30]:
delta_star(sigma_r, beta1, t_max_c, blade_type, theta, beta_prime2, a_c, C_m1, C_m2)

{'delta_m_liblin': 5.328214487756852,
 'delta_m_howel': 5.694837194288879,
 'beta2_howel': 35.69483719428888,
 'E-Check': 9.59926097010566e-06}

In [31]:
dic = delta_star(sigma_r, beta1, t_max_c, blade_type, theta, beta_prime2, a_c, C_m1, C_m2)
delta_star_r = dic['delta_m_howel']
beta2_star = dic['beta2_howel']

### loss coefficient_star

In [32]:
loss_coefficient_star = lc_star(sigma_r, r_m1, r_m2, C_m1, C_m2, N, beta1, beta2_star, H1, s_r)

In [33]:
loss_coefficient_star

0.028504228175704728

### Off-Design

For Example, We assume these informations for a an operating point:

In [34]:
N = 6000            #RPM
i = -0.7
beta1 = beta_prime1+i
sigma = sigma_r

So for rotor inlet We have:

In [35]:
df1_rotor_inlet, df2_rotor_inlet = rotor_inlet(r_h1, r_t1, N, beta1, alpha1, P01, T01, R)

In [36]:
df1_rotor_inlet

,P[bar],T[k],h[kj/kg],s[kj/kg-k]
Point 01,0.986923,288.000000,292.716203,5.866373
Point 1,0.925594,282.766378,287.461647,5.866373
Point 01_rel,1.003196,289.349486,294.071087,5.866373


In [37]:
df2_rotor_inlet

,m_dot[kg/sec],rho1[kg/m^3],C1[meter/sec],W1[meter/sec],C_m1[meter/sec],U1[meter/sec],rho1[kg/sec],A1[m^2],alpha1[degree],beta1[degree]
value,13.609998,1.155654,102.513963,114.973394,74.973967,157.079633,1.155654,0.15708,43,49.3


And for rotor outlet We have:

In [38]:
df1_rotor_outlet, df2_rotor_outlet = rotor_outlet(M_W1_star, df1_rotor_inlet, df2_rotor_inlet, gama, R, blade_type, i_s_r, i_c_r, loss_coefficient_star, sigma, beta1, delta_star_r, i_star_r, i, alpha1, r_m1, r_m2, N, beta_prime2, H1, H2, N_design, t_max_c, a_c, s_r, theta)

In [39]:
df1_rotor_outlet

,P[bar],T[k],h[kj/kg],s[kj/kg-k]
Point 02,1.042491,292.730643,297.465769,5.86701
Point 2,0.941883,284.360306,289.061951,5.86701
Point 02_rel,1.000972,289.349486,294.071087,5.86701


In [40]:
df2_rotor_outlet

,m_dot[kg/sec],C2[meter/sec],W2[meter/sec],C_m2[meter/sec],U2[meter/sec],rho2[kg/m^3],A2[m^2],alpha2[degree],beta2[degree],E-Check
value,13.609998,129.644268,100.091325,82.325011,157.079633,1.169399,0.141372,50.5795,34.664262,0.000219


## Stator Outlet

In [41]:
alpha2 = df2_rotor_outlet['alpha2[degree]']['value']
C_m2 = df2_rotor_outlet['C_m2[meter/sec]']['value']
beta2 = df2_rotor_outlet['beta2[degree]']['value']

In [42]:
i_stator = alpha2-alpha_prime2

### i_star

In [43]:
blade_type = ['C_Series','circular']
stagger = (alpha_prime2+alpha_prime3)/2
a_c = 0.5
theta = alpha_prime2-alpha_prime3

In [44]:
i_star(alpha_prime2, stagger, a_c, theta, t_max_c, sigma_s, blade_type)

{'i_star': -1.4250496630045788,
 'alpha_star': 11.07495033699542,
 'beta1_star': 56.17495033699542,
 'E-Check': 0.0}

In [45]:
dic = i_star(alpha_prime2, stagger, a_c, theta, t_max_c, sigma_s, blade_type)
i_star_s = dic['i_star']
alpha_star = dic['alpha_star']
beta1_star = dic['beta1_star']

### i_c

In [46]:
i_c(i_star_s, alpha_star, beta1_star, theta, stagger)

{'i_c': -10.196834171554006,
 'alpha_c': 2.3031658284459926,
 'beta1_c': 47.40316582844599,
 'E-Check': 1.0}

In [47]:
dic = i_c(i_star_s, alpha_star, beta1_star, theta, stagger)
i_c_s = dic['i_c']
alpha_c = dic['alpha_c']
beta1_c = dic['beta1_c']

### i_s

In [48]:
i_s(i_star_s, alpha_star, beta1_star, theta, stagger)

{'i_s': 5.454236190758787,
 'alpha_s': 17.954236190758785,
 'beta1_s': 63.05511014346361,
 'E-Check': 0.0008739527048220452}

In [49]:
dic = i_s(i_star_r, alpha_star, beta1_star, theta, stagger)
i_s_s = dic['i_s']
alpha_s = dic['alpha_s']
beta1_s = dic['beta1_s']

### delta_star

In [50]:
C_m2_star = df2_rotor_outlet_star['C_m2[meter/sec]']['value']
U2 = 2*math.pi*r_m2*N/60
C2_star = df2_rotor_outlet_star['C2[meter/sec]']['value']
T2_star = df1_rotor_outlet_star['T[k]']['Point 2']
M_C2_star = C2_star/((gama*R*1000*T2_star)**0.5)
df1_stator_outlet_star, df2_stator_outlet_star = stator_outlet(M_C2_star, df1_rotor_outlet_star, df2_rotor_outlet_star, gama, R, blade_type, i_s_s, i_c_s, 0, sigma_s, beta1_star, 0, i_star_s, i_star_s, math.degrees(math.atan(U2/C_m2_star-math.tan(math.radians(beta1_star)))), r_m2, r_m3, N_design, alpha_prime3, H2, H3, N_design, t_max_c, a_c, s_s, theta, beta1_star)

In [51]:
df1_stator_outlet_star

,P[bar],T[k],h[kj/kg],s[kj/kg-k]
Point 03,1.037099,292.598045,297.332640,5.868043
Point 3,0.950431,285.389267,290.095027,5.868043


In [52]:
df2_stator_outlet_star

,C3[meter/sec],C_m3[meter/sec],rho3[kg/m^3],A3[m^2],alpha3[degree],E-Check
value,120.313035,91.221871,1.175758,0.125664,40.693845,0.000011


In [53]:
C_m3 = df2_stator_outlet_star['C_m3[meter/sec]']['value']

In [54]:
delta_star(sigma_s, beta1_star, t_max_c, blade_type, theta, alpha_prime3, a_c, df2_rotor_outlet_star['C_m2[meter/sec]']['value'], C_m3)

{'delta_m_liblin': 9.037791725803736,
 'delta_m_howel': 8.093844720168413,
 'beta2_howel': 40.693844720168414,
 'E-Check': 6.255904828833536e-06}

In [55]:
dic = delta_star(sigma_s, beta1_star, t_max_c, blade_type, theta, alpha_prime3, a_c, df2_rotor_outlet_star['C_m2[meter/sec]']['value'], C_m3)
delta_star_s = dic['delta_m_howel']
beta2_star = dic['beta2_howel']

### lc_star

In [56]:
loss_coefficient_star = lc_star(sigma_s, r_m2, r_m3, df2_rotor_outlet_star['C_m2[meter/sec]']['value'], C_m3, N_design, beta1_star, beta2_star, H2, s_s)

In [57]:
loss_coefficient_star

0.038350320523645835

### Off-Design

In [58]:
df1_stator_outlet, df2_stator_outlet = stator_outlet(M_C2_star, df1_rotor_outlet, df2_rotor_outlet, gama, R, blade_type, i_s_s, i_c_s, loss_coefficient_star, sigma, alpha2, delta_star_s, i_star_s, i_stator, beta2, r_m2, r_m3, N, alpha_prime3, H2, H3, N_design, t_max_c, a_c, s_s, theta, beta1_star)

In [59]:
df1_stator_outlet

,P[bar],T[k],h[kj/kg],s[kj/kg-k]
Point 03,1.039135,292.730643,297.465769,5.867935
Point 3,0.955403,285.784461,290.491802,5.867935


In [60]:
df2_stator_outlet

,C3[meter/sec],C_m3[meter/sec],rho3[kg/m^3],A3[m^2],alpha3[degree],E-Check
value,118.101374,91.762447,1.180274,0.125664,39.015075,0.00009


If We have OGV We need to act like this:

## Efficiency

In [61]:
df, dic = efficiency(df1_rotor_inlet, df1_rotor_outlet, df1_stator_outlet, df2_rotor_inlet, df2_rotor_outlet, df2_stator_outlet)

In [62]:
df

,h0[kj/kg],h[kj/kg],h0_rel[kj/kg],s[kj/kg-k],P0_rel[bar],P0[bar],P[bar],C[meter/sec],C_m[meter/sec],C_theta[meter/sec],W[meter/sec],alpha[degree],beta[degree],rho[kg/m^3]
1,292.716203,287.461647,294.07108739875605,5.866373,1.0031956690742374,0.986923,0.925594,102.513963,74.973967,69.914355,114.97339393698874,43.0,49.3,1.155654
2,297.465769,289.061951,294.07108739875605,5.86701,1.000971966900706,1.042491,0.941883,129.644268,82.325011,100.151029,100.09132508145517,50.5795,34.66426213233464,1.169399
3,297.465769,290.491802,-,5.867935,-,1.039135,0.955403,118.101374,91.762447,74.347749,-,39.015075,-,1.180274


In [63]:
dic

{'m_dot[kg/sec]': 13.609997974596665,
 'etta_rotor': 0.886868538525829,
 'etta_stator': 0.8151368484405065,
 'etta_is_stage': 0.9037904508064224,
 'h01[kj/kg]': 292.71620309772544,
 'h2_s[kj/kg]': 288.88090599259436,
 'h3_s[kj/kg]': 290.22747482060345,
 'h03ss[kj/kg]': 297.0088152388821,
 'h03[kj/kg]': 297.4657688154496,
 'P03[bar]': 1.0391351263615118,
 'T03[k]': 292.73064314514363,
 'alpha3[degree]': 39.01507499972505}

## checker function

In [64]:
check, flag = checker(df, dic, i_star_r, i_star_s, delta_star_r, delta_star_s, N_stage)

everything in stage 1 is alright


In [65]:
check

,rotor,rotor_check,stator,stator_check
i_star,-0.155729,True,-1.425050,True
delta_star,5.694837,True,8.093845,True


if there is something wrong with the operating point, it will be neglected through the value of the flag(flag = 1 means that the operating point is ok!!!):

In [66]:
flag

1

## Stage Solver

Now We need to create a function that gives above results in the output:

In [67]:
i_star_r, delta_star_r, beta2_star_r, i_star_s, delta_star_s, beta2_star_s, M_W1_star, M_C2_star, i_s_r, i_c_r, i_s_s, i_c_s, loss_coefficient_star_r, loss_coefficient_star_s = stage_solver_star(
    alpha1, N_design, P01, T01, beta_prime1, beta_prime2, a_c, t_max_c, blade_type, sigma_r, sigma_s, 
    alpha_prime2, alpha_prime3, r_m1, r_m2, r_m3, H1, H2, H3, gama, R, r_h1, r_h2, r_t1, r_t2, s_s, s_r, N_stage
)

In [68]:
df1, dic1, check1, flag = stage_solver(
    alpha1, N_design, P01, T01, beta_prime1, beta_prime2, t_max_c, blade_type, a_c, i, N, sigma_r, sigma_s, alpha_prime2, alpha_prime3,
    r_m1, r_m2, r_m3, H1, H2, H3, gama, R, r_h1, r_h2, r_t1, r_t2, s_s, s_r, N_stage, i_star_r, delta_star_r, beta2_star_r, i_star_s,
    delta_star_s, beta2_star_s, M_W1_star, M_C2_star, i_s_r, i_c_r, i_s_s, i_c_s, loss_coefficient_star_r, loss_coefficient_star_s
)

everything in stage 1 is alright


In [69]:
df1

,h0[kj/kg],h[kj/kg],h0_rel[kj/kg],s[kj/kg-k],P0_rel[bar],P0[bar],P[bar],C[meter/sec],C_m[meter/sec],C_theta[meter/sec],W[meter/sec],alpha[degree],beta[degree],rho[kg/m^3]
1,292.716203,287.461647,294.07108739875605,5.866373,1.0031956690742374,0.986923,0.925594,102.513963,74.973967,69.914355,114.97339393698874,43.0,49.3,1.155654
2,297.416706,289.043024,294.0710873987561,5.867009,1.0009757360750766,1.041886,0.941668,129.411609,82.338335,99.838686,100.28024424998509,50.48713,34.80671545252108,1.16921
3,297.416706,290.436856,-,5.867929,-,1.038551,0.954784,118.151171,91.804371,74.375109,-,39.012586,-,1.179735


In [70]:
dic1

{'m_dot[kg/sec]': 13.609997974596665,
 'etta_rotor': 0.8857162862046913,
 'etta_stator': 0.8114491350686965,
 'etta_is_stage': 0.9031893036589921,
 'h01[kj/kg]': 292.71620309772544,
 'h2_s[kj/kg]': 288.86229807306415,
 'h3_s[kj/kg]': 290.1740479789004,
 'h03ss[kj/kg]': 296.96164701955075,
 'h03[kj/kg]': 297.4167059763869,
 'P03[bar]': 1.0385509284746668,
 'T03[k]': 292.68177577555923,
 'alpha3[degree]': 39.012585605704025}

In [71]:
check1

,rotor,rotor_check,stator,stator_check
i_star,0.234689,True,-1.425050,True
delta_star,5.875002,True,8.102133,True


## Compressor Solver

Now We need to write a function that solve the entire compressor at an arbitrary operating point:

In [72]:
stage1_data = (alpha1, P01, T01, beta_prime1, beta_prime2, a_c, t_max_c, blade_type, sigma_r, sigma_s, alpha_prime2,
               alpha_prime3, r_m1, r_m2, r_m3, H1, H2, H3, r_h1, r_h2, r_t1, r_t2, s_s, s_r)
compressor_data = [stage1_data]

In [73]:
P02_P01, etta_compressor, m_dot, flag = compressor_solver(compressor_data, gama, i, N, R, N_design, N_stage)

everything in stage 1 is alright
everything in compressor is ok


In [74]:
etta_compressor

0.9031893036589921

In [75]:
P02_P01

1.052311728276956

In [76]:
m_dot

13.609997974596665

So this is the full proccess, now We can plot operating curves for the given compressors, but We're going to do this work in separate files.